In [24]:
import torch
import constants.constants as c
from configs import config
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import numpy as np
import utils.utils as u
from torchvision.transforms import Normalize
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.text as text
from matplotlib.colors import ListedColormap
import glob
from PIL import Image
import pickle
from collections import Counter
import random

In [25]:
%matplotlib qt
SMALL_SIZE = 13
MEDIUM_SIZE = 15
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [26]:
u.setup_config(config)
path_images_free = ""
path_images_blocked = ""
path_patch = ""

In [27]:
images_free = glob.glob(path_images_free)
images_blocked = glob.glob(path_images_blocked)
images = []
images_ = images_free + images_blocked
random.shuffle(images_)
for i, image_name in enumerate(images_) :
    with open(image_name, "rb") as f:
        image_PIL = Image.open(f)
        images.append(np.array(image_PIL)/255)

In [28]:
trainer = pickle.load(open(path_patch,"rb"))
trainer.load(config)

In [29]:
fig = plt.figure()
subplot = fig.add_subplot()
subplot.imshow(u.tensor_to_array(trainer.best_patch))
plt.show()

In [30]:
normalize = Normalize(c.consts["NORMALIZATION_MEAN"], c.consts["NORMALIZATION_STD"])

model = u.load_model()
model.eval()

o = [None]
def save_output(module, intput, output):
    o[0] = output
    
model.classifier[5].register_forward_hook(save_output)

free = []
X = []
for image in images :
    output = model(normalize(u.array_to_tensor(image)))
    X.append(o[0].detach().numpy())
    
    model_label = torch.argmax(output, axis=1)
    free.append(int(model_label) == 1)

X = np.array(X)[:, 0, :]
pca = PCA(n_components=2)
kMeans = KMeans(n_clusters=4)
X_PCA = pca.fit_transform(X)
clusters = kMeans.fit_predict(X_PCA)
counter = Counter(clusters[free])
print(counter)
free_cluster = max(counter, key=counter.get)
for i, image in enumerate(images) :
    Image.fromarray((image * 255).astype(np.uint8)).save(
        "C:/Users/alexi/PROJET_3A/clustering_test/c%d/%d.png" % (clusters[i], i)
        )

Counter({0: 236, 3: 7})


In [31]:
def attack(trainer, image):
    attacked, vector_scores = trainer.attack_batch(image)
    vector_proba = torch.nn.functional.softmax(vector_scores, dim=1)
    target_proba = float(torch.mean(vector_proba[:, trainer.target_class]))
    model(normalize(attacked))
    point = pca.transform(o[0].detach().numpy())[0]
    return attacked, target_proba, point

In [32]:
clusters_info = {k : {"point1" : [], "point2" : [], "target_proba" : []} 
                 for k in range(4) if k!=free_cluster}
for i, image in enumerate(images) :
    if clusters[i] == free_cluster : 
        continue
    image_tensor = u.array_to_tensor(image)
    point1 = X_PCA[i, :]
    attacked, target_proba, point2 = attack(trainer, image_tensor)
    
    clusters_info[clusters[i]]["point1"].append(point1)
    clusters_info[clusters[i]]["point2"].append(point2)
    clusters_info[clusters[i]]["target_proba"].append(target_proba)

In [33]:
clusters_mean = {k : {"point1" : sum(v["point1"])/len(v["point1"]),
                      "point2" : sum(v["point2"])/len(v["point2"]),
                      "target_proba" : sum(v["target_proba"])/len(v["target_proba"])} for k, v in clusters_info.items()}
print(clusters_mean)

{1: {'point1': array([30.319077, 29.402205], dtype=float32), 'point2': array([ 3.4977415, 45.339157 ], dtype=float32), 'target_proba': 0.26035828801058875}, 2: {'point1': array([ 36.396057, -18.61073 ], dtype=float32), 'point2': array([ 2.177882, 12.679098], dtype=float32), 'target_proba': 0.2258588712213054}, 3: {'point1': array([15.155572, -7.328444], dtype=float32), 'point2': array([-19.293148,  18.213623], dtype=float32), 'target_proba': 0.7116996722826086}}


In [34]:
h = 0.02
offset1 = 5
offset20 = 17
offset21 = 12
x_min, x_max = X_PCA.min() - offset1, X_PCA.max() + offset1
y_min, y_max = X_PCA.min() - offset1, X_PCA.max() + offset1

xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
xx, yy = xx.astype(np.float32), yy.astype(np.float32)
Z = kMeans.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

fig = plt.figure(figsize=plt.figaspect(0.5))
ax1 = fig.add_subplot(1, 1, 1)

colors = ["darkorange", "gold", "lawngreen", "lightseagreen"]
cmap_ = ListedColormap(colors)

ax1.imshow(
    Z,
    interpolation="nearest",
    extent=(xx.min(), xx.max(), yy.min(), yy.max()),
    cmap=cmap_,
    aspect="auto",
    origin="lower",
)

patches_ = []
i = 0
for n, color in enumerate(colors) :
    if n == free_cluster :
        patches_.insert(0, patches.Patch(color=color, label="free")) 
    else :
        patches_.append(patches.Patch(color=color, label="blocked%d" % i)) 
        i +=1

ax1.scatter(X_PCA[:, 0], X_PCA[:, 1], label="image", c="black", alpha=0.3)

ax1.scatter(
    kMeans.cluster_centers_[:, 0],
    kMeans.cluster_centers_[:, 1],
    marker="+",
    color="black",
    s=300,
    linewidth=3,
    label="cluster center"
)

for cluster, v in clusters_mean.items():
    ax1.add_artist(patches.ConnectionPatch(v["point1"], v["point2"], coordsA="data", arrowstyle=
                                   patches.ArrowStyle("->", head_length=0.8, head_width=0.5),
                                   linewidth=3.5, color="red", zorder=200))
    ax1.add_artist(text.Text(*(v["point1"]-np.array([offset20, offset21])), "p=%.2f%%" % v["target_proba"], fontsize=40))

ax1.scatter([] , [], c="red",marker=r"$\rightarrow$", s=1000, label="average attack")
legend = ax1.legend(fontsize=20, loc=1)
ax1.add_artist(legend)
ax1.legend(handles=patches_, fontsize=20, loc=2)
plt.show()
